***Vorlesung 'Syntax natürlicher Sprachen'***

--- 
# Vorlesung 3: Syntaktische Kategorien


In [1]:
import nltk
from nltk.parse.generate import generate
import itertools
import spacy

--- 

## 1. Tagsets

---

### POS-Tagsets:

- **Universal POS-Tagset**: https://universaldependencies.org/u/pos/
  - **NLTK-Version (reduziert)**: http://www.nltk.org/book/ch05.html#tab-universal-tagset
 

- **Penn Treebank POS-Tagset**: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html


- **TIGER/STTS-POS-Tagset**: https://www.linguistik.hu-berlin.de/de/institut/professuren/korpuslinguistik/mitarbeiter-innen/hagen/STTS_Tagset_Tiger


---
### Konstituenten-Tagsets:

- **Penn Treebank Constituent Tags**: http://surdeanu.cs.arizona.edu//mihai/teaching/ista555-fall13/readings/PennTreebankConstituents.html
- **TIGER Konstituenten Labels**: https://www.linguistik.hu-berlin.de/de/institut/professuren/korpuslinguistik/mitarbeiter-innen/hagen/Tiger_Knotenlabels
 
 

---
### Dependency-Tagsets:
- **UD Dependency Labels**: https://universaldependencies.org/u/dep/all.html
- **TIGER Dependency Labels**: https://www.linguistik.hu-berlin.de/de/institut/professuren/korpuslinguistik/mitarbeiter-innen/hagen/DDB_edge
  
---

### Labels & Sources für spaCy Modelle:

- **english model**: https://github.com/explosion/spacy-models/releases/tag/en_core_web_sm-3.1.0
    - training corpus: OntoNotes (Penn-Treebank-Style-Constituents > transformation to CLEAR Dependency Labels 
    - **CLEAR Dependencies**: ähnlich **Stanford Dependencies** (Vorgänger von Universal Dependencies)

- **german model**: https://github.com/explosion/spacy-models/releases/tag/de_core_news_sm-3.1.0
    - training corpus: **TIGER** Corpus (konstituentenannotiert > dependenztransformiert = Tiger2Dep)
   
   
- in Hilfsfunktion: auch UD-Label enthalten (für andere Modelle)    

---

### Hilfsfunktionen für Erklärung von Tags

In [2]:
nltk.help.upenn_tagset('DT') #Penn-Treebank-POS

DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those


In [3]:
nltk.help.brown_tagset('DT') #Brown-POS

DT: determiner/pronoun, singular
    this each another that 'nother


In [4]:
nltk.help.brown_tagset('AT') #Brown-POS

AT: article
    the an no a every th' ever' ye


In [5]:
print(spacy.explain('ADP'), #UD-POS
      spacy.explain('IN'), #PENN-POS
      spacy.explain('APPR'), #TIGER-POS
      sep="\n")

adposition
conjunction, subordinating or preposition
preposition; circumposition left


In [6]:
print(spacy.explain('PP'), #PENN-Constituent
      spacy.explain('ADJP'),#PENN-Constituent
      spacy.explain('AP'),#TIGER-Constituent: nicht in spacy enthalten!
      sep="\n")

prepositional phrase
adjective phrase
None


In [7]:
print(spacy.explain('nk'), #TIGER-Dep
      spacy.explain('case'), #UD-Dep
      sep="\n")

noun kernel element
case marking


---

## 2. Generierung Satzschemata (POS-Muster) mit Phrasenstruktur-Regeln

In [8]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    VP -> 'V' | 'V' NP | 'V' NP NP
    NP -> 'DET' 'N' | 'N'
""")

#Generierung (http://www.nltk.org/howto/generate.html):
from nltk.parse.generate import generate
print('Anzahl an POS-Mustern: ', 
    '\n\tbei 3 Regelanwendungen:', len(list(generate(grammar, depth=3))), 
    '\n\tbei 4 Regelanwendungen:', len(list(generate(grammar, depth=4))),
    '\n\tbei 5 Regelanwendungen:', len(list(generate(grammar, depth=5))))                                                     
                                                     

Anzahl an POS-Mustern:  
	bei 3 Regelanwendungen: 2 
	bei 4 Regelanwendungen: 14 
	bei 5 Regelanwendungen: 14


In [9]:
for sentence in generate(grammar, depth=3):
    print(' '.join(sentence))

DET N V
N V


In [10]:
for sentence in generate(grammar, depth=4):
    print(' '.join(sentence))

DET N V
DET N V DET N
DET N V N
DET N V DET N DET N
DET N V DET N N
DET N V N DET N
DET N V N N
N V
N V DET N
N V N
N V DET N DET N
N V DET N N
N V N DET N
N V N N


### Generierung Satzschemata (POS-Muster) mit rekursiver Phrasenstruktur-Regel:

In [11]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> NP PP
    PP -> 'P' NP
    NP -> 'DET' 'N'
    NP -> 'N'
    VP -> 'V' NP 
""")

#Generierung:
from nltk.parse.generate import generate
print('Anzahl an POS-Mustern: ', 
    '\n\tbei 6 Regelanwendungen:', len(list(generate(grammar, depth=6))), 
    '\n\tbei 7 Regelanwendungen:', len(list(generate(grammar, depth=7))),
    '\n\tbei 8 Regelanwendungen:', len(list(generate(grammar, depth=8))))

Anzahl an POS-Mustern:  
	bei 6 Regelanwendungen: 84 
	bei 7 Regelanwendungen: 1204 
	bei 8 Regelanwendungen: 103716


In [12]:
for sentence in generate(grammar, depth=6):
    print(' '.join(sentence))

DET N P DET N P DET N V DET N P DET N
DET N P DET N P DET N V DET N P N
DET N P DET N P DET N V N P DET N
DET N P DET N P DET N V N P N
DET N P DET N P DET N V DET N
DET N P DET N P DET N V N
DET N P DET N P N V DET N P DET N
DET N P DET N P N V DET N P N
DET N P DET N P N V N P DET N
DET N P DET N P N V N P N
DET N P DET N P N V DET N
DET N P DET N P N V N
DET N P N P DET N V DET N P DET N
DET N P N P DET N V DET N P N
DET N P N P DET N V N P DET N
DET N P N P DET N V N P N
DET N P N P DET N V DET N
DET N P N P DET N V N
DET N P N P N V DET N P DET N
DET N P N P N V DET N P N
DET N P N P N V N P DET N
DET N P N P N V N P N
DET N P N P N V DET N
DET N P N P N V N
N P DET N P DET N V DET N P DET N
N P DET N P DET N V DET N P N
N P DET N P DET N V N P DET N
N P DET N P DET N V N P N
N P DET N P DET N V DET N
N P DET N P DET N V N
N P DET N P N V DET N P DET N
N P DET N P N V DET N P N
N P DET N P N V N P DET N
N P DET N P N V N P N
N P DET N P N V DET N
N P DET N P N V N
N P N P DET N V 

---

## 3. Distributionsanalyse (Feststellung von Wortklassen)

### Suche kontextäquivalenter Wörter im Korpus mit NLTK (paradigmatische Dimension)

http://www.nltk.org/book/ch05.html#using-a-tagger:

> Lexical categories like "noun" and part-of-speech tags like NN seem to have their uses, but the details will be obscure to many readers. You might wonder what justification there is for introducing this extra level of information. Many of these categories arise from superficial analysis the distribution of words in text. Consider the following analysis involving *woman* (a noun), *bought* (a verb), *over* (a preposition), and *the* (a determiner). The `text.similar()` method takes a word *w*, finds all contexts *w1 w w2*, then finds all words *w'* that appear in the same context, i.e. *w1 w' w2*.



In [13]:
#http://www.nltk.org/book/ch05.html#using-a-tagger

from nltk.corpus import brown
text = nltk.Text(word.lower() for word in nltk.corpus.brown.words())

> Observe that searching for *woman* finds nouns; searching for *bought* mostly finds verbs; searching for *over* generally finds prepositions; searching for *the* finds several determiners. A tagger can correctly identify the tags on these words in the context of a sentence, e.g. *The woman bought over $150,000 worth of clothes.* (http://www.nltk.org/book/ch05.html#using-a-tagger)



In [14]:
text.similar('woman')

man time day year car moment world house family child country boy
state job place way war girl work word


In [15]:
text.similar('bought')

made said done put had seen found given left heard was been brought
set got that took in told felt


In [16]:
text.similar('over')

in on to of and for with from at by that into as up out down through
is all about


In [17]:
text.similar('the')

a his this their its her an that our any all one these my in your no
some other and


--- 

## 4. Distribution von Konstituenten

### Suche nach nominalen POS-Mustern (Syntagmen) im Korpus  

http://www.nltk.org/book/ch05.html#nouns:

> Let's inspect some tagged text to see what parts of speech occur before a noun, with the most frequent ones first. To begin with, we construct a list of bigrams whose members are themselves word-tag pairs such as `(('The', 'DET'), ('Fulton', 'NP'))` and `(('Fulton', 'NP'), ('County', 'N'))`. Then we construct a `FreqDist` from the tag parts of the bigrams.


In [18]:
from nltk.corpus import brown
brown_news_tagged = brown.tagged_words(categories='news', tagset='universal')
word_tag_pairs = nltk.bigrams(brown_news_tagged)
noun_preceders = [a[1] for (a, b) in word_tag_pairs if b[1] == 'NOUN']
fdist = nltk.FreqDist(noun_preceders)
[(tag, fq) for (tag, fq) in fdist.most_common()]

[('NOUN', 7959),
 ('DET', 7373),
 ('ADJ', 4761),
 ('ADP', 3781),
 ('.', 2796),
 ('VERB', 1842),
 ('CONJ', 938),
 ('NUM', 894),
 ('ADV', 186),
 ('PRT', 94),
 ('PRON', 19),
 ('X', 11)]

> This confirms our assertion that nouns occur after determiners and adjectives, including numeral adjectives (tagged as `NUM``). (http://www.nltk.org/book/ch05.html#nouns)


--- 

### Adjektive als Klasse distributionsäquivalenter Wörter:

#### Suche nach distributionsäquivalenten Wörtern (Auftreten in gleichen Kontexten):

In [19]:
from nltk.corpus import brown
text = nltk.Text(word.lower() for word in nltk.corpus.brown.words())
text.similar('big')

little new first good small large great the old other strong young
major white second short beautiful a best long


#### Wortarten-Kontexte von distributionsäquivalenter Wörtern (als Vertreter einer Distributionsklasse):

In [20]:
#Rechter und linker Kontext für eine Menge distributionsäquivalenter Wörter:
from nltk.corpus import brown
brown_news_tagged = brown.tagged_words(categories='news', tagset='universal')
word_tag_trigrams = nltk.trigrams(brown_news_tagged)
adj_contexts = [(a[1], c[1]) for (a, b, c) in word_tag_trigrams if b[0] in ('big', 'little', 'new', 'first', 'good', 'small', 'large', 'great')]

fdist = nltk.FreqDist(adj_contexts)
[(tag, fq) for (tag, fq) in fdist.most_common(20)]

[(('DET', 'NOUN'), 193),
 (('ADP', 'NOUN'), 42),
 (('VERB', 'NOUN'), 36),
 (('DET', 'ADJ'), 30),
 (('DET', 'NUM'), 22),
 (('NOUN', 'NOUN'), 13),
 (('DET', 'ADP'), 13),
 (('ADJ', 'NOUN'), 12),
 (('CONJ', 'NOUN'), 10),
 (('NUM', 'NOUN'), 10),
 (('NOUN', 'ADJ'), 8),
 (('ADV', 'NOUN'), 8),
 (('VERB', '.'), 7),
 (('.', 'NOUN'), 6),
 (('DET', '.'), 5),
 (('ADV', 'ADP'), 5),
 (('ADV', '.'), 5),
 (('NOUN', '.'), 4),
 (('VERB', 'VERB'), 4),
 (('DET', 'VERB'), 4)]

In [21]:
#Linker und rechter Kontext für ADJ:
from nltk.corpus import brown
brown_news_tagged = brown.tagged_words(categories='news', tagset='universal')
word_tag_trigrams = nltk.trigrams(brown_news_tagged)
adj_contexts = [(a[1], c[1]) for (a, b, c) in word_tag_trigrams if b[1] == 'ADJ']

fdist = nltk.FreqDist(adj_contexts)
[(tag, fq) for (tag, fq) in fdist.most_common(20)]

[(('DET', 'NOUN'), 2081),
 (('ADP', 'NOUN'), 745),
 (('NOUN', 'NOUN'), 368),
 (('ADJ', 'NOUN'), 363),
 (('VERB', 'NOUN'), 351),
 (('.', 'NOUN'), 332),
 (('DET', 'ADJ'), 218),
 (('CONJ', 'NOUN'), 214),
 (('ADV', 'NOUN'), 154),
 (('VERB', 'ADP'), 145),
 (('NUM', 'NOUN'), 129),
 (('DET', '.'), 104),
 (('ADV', 'ADP'), 100),
 (('ADV', '.'), 83),
 (('DET', 'NUM'), 83),
 (('VERB', '.'), 77),
 (('VERB', 'PRT'), 61),
 (('.', 'ADP'), 57),
 (('DET', 'CONJ'), 56),
 (('NOUN', 'ADP'), 56)]

---

## 5. Konstituententests
---

### Konstituententests 1: Eliminierung

####  vor allem zur Feststellung nicht-notwendiger Elemente

##### Kriterium: Erhalt der Wohlgeformtheit bei Löschung (= Substitution mit leerem Wort)

---

In [2]:
sentence = ["der", "sehr große", "Hund"]

sentencelist = []
for i in range(len((sentence))):
    sentencelist.append(sentence.copy())
    sentencelist[i].pop(i)
    print(i, sentencelist[i])

0 ['sehr große', 'Hund']
1 ['der', 'Hund']
2 ['der', 'sehr große']


In [3]:
#Erhalt der Wohlgeformtheit:
print(sentencelist[1])
    # 'sehr große' als weglassbare Konstituente (ADJ-Phrase ADJP)

['der', 'Hund']


---

### Konstituententests 2: Substitution

#### Substituierbarkeit als Evidenz für Vorliegen einer syntaktischen Einheit gleicher Kategorie (Konstituente)

   ##### Austauschbarkeit im gleichen Kontext

##### Kriterium: Erhalt der Wohlgeformtheit bei Ersatz mit kürzerer Einheit 
- insbesondere Pronomen bei NPs, intransitiven Verben bei VPs


---

In [4]:
# Test: Ersatz VP mit intransitivem Verb
sentence = ["ich", "sehe nichts"]
substitute = "laufe"

sentencelist = []
for i in range(len((sentence))):
    sentencelist.append(sentence.copy())
    sentencelist[i][i] = substitute
    print(i, sentencelist[i])

0 ['laufe', 'sehe nichts']
1 ['ich', 'laufe']


In [5]:
# Erhalt der Wohlgeformtheit:
print( sentencelist[1])
    #'sehe nichts' als Konstituente (VP)

['ich', 'laufe']


---

### Konstituententests 3: Permutation

#### Verschiebbarkeit als Evidenz für Vorliegen einer syntaktischen Einheit (Konstituente)


##### Kriterien für Feststellung als syntaktische Einheit:

1. Wortgruppe an anderer Position
2. Erhalt der Wohlgeformtheit


---

In [26]:
sentence = ["der Mann", "sieht", "nichts"]

permutations = list(itertools.permutations(sentence))
for (i, item) in enumerate(permutations):
    print(i, item)    

0 ('der Mann', 'sieht', 'nichts')
1 ('der Mann', 'nichts', 'sieht')
2 ('sieht', 'der Mann', 'nichts')
3 ('sieht', 'nichts', 'der Mann')
4 ('nichts', 'der Mann', 'sieht')
5 ('nichts', 'sieht', 'der Mann')


In [27]:
#wohlgeformte Sätze:
print(list(itertools.permutations(sentence))[0])
print(list(itertools.permutations(sentence))[1], "IM NEBENSATZKONTEXT")
print(list(itertools.permutations(sentence))[2], "IM FRAGEKONTEXT")
print(list(itertools.permutations(sentence))[5])

('der Mann', 'sieht', 'nichts')
('der Mann', 'nichts', 'sieht') IM NEBENSATZKONTEXT
('sieht', 'der Mann', 'nichts') IM FRAGEKONTEXT
('nichts', 'sieht', 'der Mann')


In [28]:
#davon Permutationen, die die NP ('der Mann') als Konstituente bestätigen:

print("NEIN (keine Permutation):", list(itertools.permutations(sentence))[0])
print("NEIN (keine Permutation der NP):", list(itertools.permutations(sentence))[1])
print("JA:", list(itertools.permutations(sentence))[2])
print("JA:", list(itertools.permutations(sentence))[5])

NEIN (keine Permutation): ('der Mann', 'sieht', 'nichts')
NEIN (keine Permutation der NP): ('der Mann', 'nichts', 'sieht')
JA: ('sieht', 'der Mann', 'nichts')
JA: ('nichts', 'sieht', 'der Mann')


In [29]:
#Permutationen, die die VP ('sieht nichts') als Konstituente bestätigen:

print("NEIN (keine Permutation):", list(itertools.permutations(sentence))[0])
print("NEIN (Wortgruppe nicht gemeinsam verschoben):", list(itertools.permutations(sentence))[1])
print("NEIN (Wortgruppe nicht gemeinsam verschoben):", list(itertools.permutations(sentence))[2])
print("NEIN (Wortgruppe nicht gemeinsam verschoben):", list(itertools.permutations(sentence))[5])

NEIN (keine Permutation): ('der Mann', 'sieht', 'nichts')
NEIN (Wortgruppe nicht gemeinsam verschoben): ('der Mann', 'nichts', 'sieht')
NEIN (Wortgruppe nicht gemeinsam verschoben): ('sieht', 'der Mann', 'nichts')
NEIN (Wortgruppe nicht gemeinsam verschoben): ('nichts', 'sieht', 'der Mann')
